# The Currencies data-set

The python version of this sample can be found [here](https://github.com/shyams80/plutons/blob/master/docs-py/Currencies.ipynb)

In [1]:
library(dbplyr)
library(dplyr)
library(odbc)
library(RPostgres)
library(plutoR)
options("scipen"=999)
source("config.R")

#initialize
currencies <- Currencies()


Attaching package: ‘dplyr’

The following objects are masked from ‘package:dbplyr’:

    ident, sql

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



### get traded futures pairs

In [2]:
currencies$NseFuturesTimeSeries() %>%
  group_by(SYMBOL) %>%
  summarize(START_DT = min(TIME_STAMP), END_DT = max(TIME_STAMP)) %>%
  arrange(START_DT) %>%
  print()

Warning message:
“Missing values are always removed in SQL.
Use `MIN(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”Warning message:
“Missing values are always removed in SQL.
Use `MAX(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

# Source:     lazy query [?? x 3]
# Database:   Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
# Ordered by: START_DT
  SYMBOL START_DT   END_DT    
  <chr>  <date>     <date>    
1 GBPINR 2010-10-29 2019-07-22
2 USDINR 2010-10-29 2019-07-22
3 JPYINR 2010-10-29 2019-07-22
4 EURINR 2010-10-29 2019-07-22
5 USDJPY 2018-02-27 2019-07-22
6 GBPUSD 2018-02-27 2019-07-22
7 EURUSD 2018-02-27 2019-07-22


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)